# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 1:51PM,243791,10,8381345,Yusen – 3D Matrix,Released
1,Aug 9 2022 1:19PM,243787,10,8380759,Beach Products Inc,Released
2,Aug 9 2022 1:19PM,243787,10,8381289,Beach Products Inc,Released
3,Aug 9 2022 1:19PM,243786,10,SO94344,"Senseonics, Incorporated",Released
4,Aug 9 2022 1:19PM,243786,10,SO94345,"Senseonics, Incorporated",Released
5,Aug 9 2022 1:19PM,243786,10,SO94346,"Senseonics, Incorporated",Released
6,Aug 9 2022 1:17PM,243785,10,Enova-10445,Emerginnova,Released
7,Aug 9 2022 1:07PM,243783,12,HH6539,Hush Hush,Released
8,Aug 9 2022 12:46PM,243777,10,MSP212976,"Methapharm, Inc.",Executing
9,Aug 9 2022 12:46PM,243777,10,MSP212983,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243783,Released,1
31,243785,Released,1
32,243786,Released,3
33,243787,Released,2
34,243791,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243783,NaN,NaN,1.0
243785,NaN,NaN,1.0
243786,NaN,NaN,3.0
243787,NaN,NaN,2.0
243791,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243687,13.0,1.0,1.0
243722,0.0,0.0,1.0
243723,13.0,25.0,70.0
243724,0.0,21.0,18.0
243732,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243687,13,1,1
243722,0,0,1
243723,13,25,70
243724,0,21,18
243732,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243687,13,1,1
1,243722,0,0,1
2,243723,13,25,70
3,243724,0,21,18
4,243732,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243687,13,1,1
1,243722,,,1
2,243723,13,25,70
3,243724,,21,18
4,243732,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc
3,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated"
6,Aug 9 2022 1:17PM,243785,10,Emerginnova
7,Aug 9 2022 1:07PM,243783,12,Hush Hush
8,Aug 9 2022 12:46PM,243777,10,"Methapharm, Inc."
10,Aug 9 2022 12:41PM,243778,20,"ACI Healthcare USA, Inc."
39,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc."
45,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation
51,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix,,,1
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc,,,2
2,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated",,,3
3,Aug 9 2022 1:17PM,243785,10,Emerginnova,,,1
4,Aug 9 2022 1:07PM,243783,12,Hush Hush,,,1
5,Aug 9 2022 12:46PM,243777,10,"Methapharm, Inc.",,1,1
6,Aug 9 2022 12:41PM,243778,20,"ACI Healthcare USA, Inc.",,3,26
7,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",,,6
8,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,,,6
9,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",2,16,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix,1,,
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc,2,,
2,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated",3,,
3,Aug 9 2022 1:17PM,243785,10,Emerginnova,1,,
4,Aug 9 2022 1:07PM,243783,12,Hush Hush,1,,
5,Aug 9 2022 12:46PM,243777,10,"Methapharm, Inc.",1,1,
6,Aug 9 2022 12:41PM,243778,20,"ACI Healthcare USA, Inc.",26,3,
7,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",6,,
8,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,6,,
9,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",7,16,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix,1,,
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc,2,,
2,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated",3,,
3,Aug 9 2022 1:17PM,243785,10,Emerginnova,1,,
4,Aug 9 2022 1:07PM,243783,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc,2.0,NaN,NaN
2,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated",3.0,NaN,NaN
3,Aug 9 2022 1:17PM,243785,10,Emerginnova,1.0,NaN,NaN
4,Aug 9 2022 1:07PM,243783,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 1:51PM,243791,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Aug 9 2022 1:19PM,243787,10,Beach Products Inc,2.0,0.0,0.0
2,Aug 9 2022 1:19PM,243786,10,"Senseonics, Incorporated",3.0,0.0,0.0
3,Aug 9 2022 1:17PM,243785,10,Emerginnova,1.0,0.0,0.0
4,Aug 9 2022 1:07PM,243783,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2681431,36.0,34.0,1.0
12,243783,1.0,0.0,0.0
15,731202,96.0,46.0,13.0
19,487517,8.0,16.0,2.0
20,974982,33.0,19.0,13.0
21,243722,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2681431,36.0,34.0,1.0
1,12,243783,1.0,0.0,0.0
2,15,731202,96.0,46.0,13.0
3,19,487517,8.0,16.0,2.0
4,20,974982,33.0,19.0,13.0
5,21,243722,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,34.0,1.0
1,12,1.0,0.0,0.0
2,15,96.0,46.0,13.0
3,19,8.0,16.0,2.0
4,20,33.0,19.0,13.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,12,Released,1.0
2,15,Released,96.0
3,19,Released,8.0
4,20,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,1.0,0.0,13.0,2.0,13.0,0.0
Executing,34.0,0.0,46.0,16.0,19.0,0.0
Released,36.0,1.0,96.0,8.0,33.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,13.0,2.0,13.0,0.0
1,Executing,34.0,0.0,46.0,16.0,19.0,0.0
2,Released,36.0,1.0,96.0,8.0,33.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,13.0,2.0,13.0,0.0
1,Executing,34.0,0.0,46.0,16.0,19.0,0.0
2,Released,36.0,1.0,96.0,8.0,33.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()